In [3]:
import requests
import warnings
import time
from tqdm import tqdm
import pandas as pd
from datasets import Dataset
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from unidecode import unidecode
import re      #  để làm việc với biểu thức chính quy (regular expressions), để xử lý,thao tác chuỗi theo các pattern cụ thể
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

D:\envs\tf26\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd

In [5]:
a = pd.read_csv("./dataset.txt", on_bad_lines='skip', header=None)

In [6]:
a.head(50)

,0
0,nước ép dưa hấu siêu ngọt khổng lồ||smoothie d...
1,nuoc ep dua hau sieu ngot khong lo||smoothie d...
2,nước ép dưa hấu siêu ngọt khổng lồ||cà phê sữa...
3,nuoc ep dua hau sieu ngot khong lo||cà phê sữa...
4,nước ép dưa hấu siêu ngọt khổng lồ||nước ép th...
5,nuoc ep dua hau sieu ngot khong lo||nước ép th...
6,nước ép dưa hấu siêu ngọt khổng lồ||khay dưa h...
7,nuoc ep dua hau sieu ngot khong lo||khay dưa h...
8,nước ép dưa hấu siêu ngọt khổng lồ||cơm gà xối...
9,nuoc ep dua hau sieu ngot khong lo||cơm gà xối...


In [7]:
df = pd.read_csv("./dataset.txt", sep='|', on_bad_lines='skip', header=None)
df = df[[0,2,4]]
df.columns = ['description','name','labels']
df['labels'] = df['labels'].astype(int)

In [8]:
pd.options.display.max_colwidth = 2000

In [9]:
df

,description,name,labels
0,nước ép dưa hấu siêu ngọt khổng lồ,smoothie dưa hấu,1
1,nuoc ep dua hau sieu ngot khong lo,smoothie dưa hấu,1
2,nước ép dưa hấu siêu ngọt khổng lồ,cà phê sữa gấu,0
3,nuoc ep dua hau sieu ngot khong lo,cà phê sữa gấu,0
4,nước ép dưa hấu siêu ngọt khổng lồ,nước ép thơm nguyên chất chai,1
...,...,...,...
1356855,rtc canh ga sot ro ti khay,bánh tráng cuốn sốt me bơ,0
1356856,súp rau củ trứng cút,trứng cút rim bơ tỏi,1
1356857,sup rau cu trung cut,trứng cút rim bơ tỏi,1
1356858,súp rau củ trứng cút,chả cả thêm,0


In [10]:
from sklearn.model_selection import train_test_split
df, remove = train_test_split(df, test_size=0.95, random_state=42)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
device

device(type='cuda')

In [13]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [24]:
# Custom Dataset class
class VietnameseDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Tokenization function
def preprocess_function(descriptions, names):
    combined_texts = descriptions + " [SEP] " + names
    return tokenizer(combined_texts.tolist(), padding="max_length", truncation=True, max_length=64, return_tensors="pt")

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df[["description", "name"]], df["labels"], test_size=0.2, random_state=42, stratify=df["labels"].values.tolist()
)

# Tokenize training and validation data
train_encodings = preprocess_function(train_texts["description"], train_texts["name"])
val_encodings = preprocess_function(val_texts["description"], val_texts["name"])

# Create Dataset objects
train_dataset = VietnameseDataset(train_encodings, train_labels.tolist())
val_dataset = VietnameseDataset(val_encodings, val_labels.tolist())

In [15]:
from transformers import RobertaModel, AutoTokenizer, AdamW
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

from tqdm import tqdm
import time

from tqdm import tqdm
import time

class PhoBERTWithClassification(torch.nn.Module):
    def __init__(self):
        super(PhoBERTWithClassification, self).__init__()
        self.phobert = RobertaModel.from_pretrained("vinai/phobert-base")
        self.linear = torch.nn.Linear(768, 768)
        self.activation = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        # PhoBERT does not use `token_type_ids`
        output_with_pooling = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_with_pooling[0]
        pooler = hidden_state[:, 0]
        pooler = self.linear(pooler)
        pooler = self.activation(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

class PhoBERTTrainer:
    def __init__(self, model, tokenizer, train_dataset, val_dataset, batch_size=16, lr=5e-5, device='cuda'):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        self.val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
        self.optimizer = AdamW(model.parameters(), lr=lr)
        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.device = device

    def train_epoch(self, epoch, total_epochs):
        self.model.train()
        total_loss = 0

        # Initialize tqdm progress bar (dynamic update frequency)
        progress_bar = tqdm(
            self.train_dataloader,
            desc=f"Epoch {epoch+1}/{total_epochs} - Training",
            unit="batch",
            leave=False,
            miniters=50,  # Update bar every 10 iterations (reduces CPU load)
        )
        start_time = time.time()
        total_batches = len(self.train_dataloader)

        for batch_idx, batch in enumerate(self.train_dataloader):
            input_ids = batch['input_ids'].to(self.device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(self.device, non_blocking=True)
            labels = batch['labels'].to(self.device, non_blocking=True)

            self.optimizer.zero_grad()
            outputs = self.model(input_ids, attention_mask)
            loss = self.loss_fn(outputs, labels)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()

            # Update progress bar only every `miniters`
            if batch_idx % progress_bar.miniters == 0 or batch_idx == total_batches - 1:
                elapsed_time = time.time() - start_time
                batches_done = batch_idx + 1
                batches_left = total_batches - batches_done
                time_left = elapsed_time / batches_done * batches_left

                progress_bar.set_postfix(
                    {
                        "Batch Loss": f"{loss.item():.4f}",
                        "Avg Loss": f"{total_loss / batches_done:.4f}",
                        "Time Left": f"{time_left / 60:.2f} min",
                    }
                )
                progress_bar.update(progress_bar.miniters)  # Advance the progress bar by `miniters`
        return total_loss / len(self.train_dataloader)

    def evaluate(self, epoch, total_epochs):
        self.model.eval()
        total_loss = 0
        preds, true_labels = [], []

        # Initialize tqdm progress bar for evaluation
        progress_bar = tqdm(
            self.val_dataloader,
            desc=f"Epoch {epoch+1}/{total_epochs} - Evaluating",
            unit="batch",
            leave=False,
        )

        with torch.no_grad():
            for batch in self.val_dataloader:
                input_ids = batch['input_ids'].to(self.device, non_blocking=True)
                attention_mask = batch['attention_mask'].to(self.device, non_blocking=True)
                labels = batch['labels'].to(self.device, non_blocking=True)

                outputs = self.model(input_ids, attention_mask)
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()

                preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(true_labels, preds)
        return total_loss / len(self.val_dataloader), accuracy

    def fine_tune(self, epochs=3):
        for epoch in range(epochs):
            print(f"Starting Epoch {epoch+1}/{epochs}")
            
            train_loss = self.train_epoch(epoch, epochs)
            val_loss, val_accuracy = self.evaluate(epoch, epochs)

            print(f"\nEpoch {epoch+1}/{epochs} Results:")
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}\n")

In [16]:
model = PhoBERTWithClassification()

In [125]:
# Initialize the PhoBERTTrainer
trainer = PhoBERTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=16,
    lr=2e-5,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Fine-tune the model
trainer.fine_tune(epochs=3)

# Save the fine-tuned model
model.phobert.save_pretrained("./phobert-finetuned-vietnamese")
tokenizer.save_pretrained("./phobert-finetuned-vietnamese")

D:\envs\tf26\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Starting Epoch 1/3



Epoch 1/3 - Training:   0%|                                                                | 0/3393 [00:00<?, ?batch/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_14248\3382488418.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

Epoch 1/3 - Training:   0%|       | 0/3393 [00:00<?, ?batch/s, Batch Loss=0.7031, Avg Loss=0.7031, Time Left=30.21 min]
Epoch 1/3 - Training:   3%| | 100/3393 [00:00<00:17, 187.15batch/s, Batch Loss=0.7031, Avg Loss=0.7031, Time Left=30.21
Epoch 1/3 - Training:   3%| | 100/3393 [00:11<00:17, 187.15batch/s, Batch Loss=0.7031, Avg Loss=0.7031, Time Left=30.21
Epoch 1/3 - Training:   3%| | 100/3393 [00:11<00:17, 187.15batch/s, Batch Loss=0.6931, Avg Loss=0.6948, Time Left=25.18
Epoch 1/3 - Training:   3%| | 101/3393 [00:11<08:32,  6.4

Epoch 1/3 - Training:   4%| | 130/3393 [00:25<26:28,  2.05batch/s, Batch Loss=0.6965, Avg Loss=0.6921, Time Left=25.90 
Epoch 1/3 - Training:   4%| | 130/3393 [00:25<26:28,  2.05batch/s, Batch Loss=0.6528, Avg Loss=0.6914, Time Left=25.89 
Epoch 1/3 - Training:   4%| | 131/3393 [00:25<26:11,  2.08batch/s, Batch Loss=0.6528, Avg Loss=0.6914, Time Left=25.89 
Epoch 1/3 - Training:   4%| | 131/3393 [00:26<26:11,  2.08batch/s, Batch Loss=0.6710, Avg Loss=0.6910, Time Left=25.93 
Epoch 1/3 - Training:   4%| | 132/3393 [00:26<26:40,  2.04batch/s, Batch Loss=0.6710, Avg Loss=0.6910, Time Left=25.93 
Epoch 1/3 - Training:   4%| | 132/3393 [00:26<26:40,  2.04batch/s, Batch Loss=0.6814, Avg Loss=0.6908, Time Left=25.93 
Epoch 1/3 - Training:   4%| | 133/3393 [00:26<26:21,  2.06batch/s, Batch Loss=0.6814, Avg Loss=0.6908, Time Left=25.93 
Epoch 1/3 - Training:   4%| | 133/3393 [00:27<26:21,  2.06batch/s, Batch Loss=0.6709, Avg Loss=0.6905, Time Left=25.96 
Epoch 1/3 - Training:   4%| | 134/3393 [

Epoch 1/3 - Training:   5%| | 163/3393 [00:41<26:01,  2.07batch/s, Batch Loss=0.6688, Avg Loss=0.6852, Time Left=26.43 
Epoch 1/3 - Training:   5%| | 163/3393 [00:42<26:01,  2.07batch/s, Batch Loss=0.6218, Avg Loss=0.6845, Time Left=26.45 
Epoch 1/3 - Training:   5%| | 164/3393 [00:42<26:33,  2.03batch/s, Batch Loss=0.6218, Avg Loss=0.6845, Time Left=26.45 
Epoch 1/3 - Training:   5%| | 164/3393 [00:42<26:33,  2.03batch/s, Batch Loss=0.6264, Avg Loss=0.6838, Time Left=26.43 
Epoch 1/3 - Training:   5%| | 165/3393 [00:42<26:19,  2.04batch/s, Batch Loss=0.6264, Avg Loss=0.6838, Time Left=26.43 
Epoch 1/3 - Training:   5%| | 165/3393 [00:43<26:19,  2.04batch/s, Batch Loss=0.6349, Avg Loss=0.6833, Time Left=26.42 
Epoch 1/3 - Training:   5%| | 166/3393 [00:43<25:43,  2.09batch/s, Batch Loss=0.6349, Avg Loss=0.6833, Time Left=26.42 
Epoch 1/3 - Training:   5%| | 166/3393 [00:43<25:43,  2.09batch/s, Batch Loss=0.6096, Avg Loss=0.6825, Time Left=26.43 
Epoch 1/3 - Training:   5%| | 167/3393 [

...

Epoch 3/3 - Training:  99%|▉| 3364/3393 [28:05<00:15,  1.87batch/s, Batch Loss=0.2126, Avg Loss=0.0741, Time Left=0.81 
Epoch 3/3 - Training:  99%|▉| 3364/3393 [28:06<00:15,  1.87batch/s, Batch Loss=0.0703, Avg Loss=0.0741, Time Left=0.80 
Epoch 3/3 - Training:  99%|▉| 3365/3393 [28:06<00:14,  1.95batch/s, Batch Loss=0.0703, Avg Loss=0.0741, Time Left=0.80 
Epoch 3/3 - Training:  99%|▉| 3365/3393 [28:06<00:14,  1.95batch/s, Batch Loss=0.1216, Avg Loss=0.0741, Time Left=0.79 
Epoch 3/3 - Training:  99%|▉| 3366/3393 [28:06<00:13,  1.98batch/s, Batch Loss=0.1216, Avg Loss=0.0741, Time Left=0.79 
Epoch 3/3 - Training:  99%|▉| 3366/3393 [28:07<00:13,  1.98batch/s, Batch Loss=0.0323, Avg Loss=0.0741, Time Left=0.78 
Epoch 3/3 - Training:  99%|▉| 3367/3393 [28:07<00:13,  1.97batch/s, Batch Loss=0.0323, Avg Loss=0.0741, Time Left=0.78 
Epoch 3/3 - Training:  99%|▉| 3367/3393 [28:07<00:13,  1.97batch/s, Batch Loss=0.0838, Avg Loss=0.0741, Time Left=0.78 
Epoch 3/3 - Training:  99%|▉| 3368/3393 

Epoch 3/3 - Training: 3397batch [28:22,  2.00batch/s, Batch Loss=0.1044, Avg Loss=0.0741, Time Left=0.52 min]
Epoch 3/3 - Training: 3398batch [28:22,  2.00batch/s, Batch Loss=0.1044, Avg Loss=0.0741, Time Left=0.52 min]
Epoch 3/3 - Training: 3398batch [28:23,  2.00batch/s, Batch Loss=0.1128, Avg Loss=0.0741, Time Left=0.51 min]
Epoch 3/3 - Training: 3399batch [28:23,  1.98batch/s, Batch Loss=0.1128, Avg Loss=0.0741, Time Left=0.51 min]
Epoch 3/3 - Training: 3399batch [28:24,  1.98batch/s, Batch Loss=0.0020, Avg Loss=0.0741, Time Left=0.50 min]
Epoch 3/3 - Training: 3400batch [28:24,  1.97batch/s, Batch Loss=0.0020, Avg Loss=0.0741, Time Left=0.50 min]
Epoch 3/3 - Training: 3400batch [28:24,  1.97batch/s, Batch Loss=0.2448, Avg Loss=0.0741, Time Left=0.49 min]
Epoch 3/3 - Training: 3401batch [28:24,  2.00batch/s, Batch Loss=0.2448, Avg Loss=0.0741, Time Left=0.49 min]
Epoch 3/3 - Training: 3401batch [28:24,  2.00batch/s, Batch Loss=0.0574, Avg Loss=0.0741, Time Left=0.49 min]
Epoch 3/3 

Epoch 3/3 - Training: 3433batch [28:41,  1.96batch/s, Batch Loss=0.0038, Avg Loss=0.0739, Time Left=0.21 min]
Epoch 3/3 - Training: 3434batch [28:41,  1.97batch/s, Batch Loss=0.0038, Avg Loss=0.0739, Time Left=0.21 min]
Epoch 3/3 - Training: 3434batch [28:41,  1.97batch/s, Batch Loss=0.0010, Avg Loss=0.0738, Time Left=0.20 min]
Epoch 3/3 - Training: 3435batch [28:41,  2.00batch/s, Batch Loss=0.0010, Avg Loss=0.0738, Time Left=0.20 min]
Epoch 3/3 - Training: 3435batch [28:42,  2.00batch/s, Batch Loss=0.0066, Avg Loss=0.0738, Time Left=0.20 min]
Epoch 3/3 - Training: 3436batch [28:42,  1.94batch/s, Batch Loss=0.0066, Avg Loss=0.0738, Time Left=0.20 min]
Epoch 3/3 - Training: 3436batch [28:42,  1.94batch/s, Batch Loss=0.0284, Avg Loss=0.0738, Time Left=0.19 min]
Epoch 3/3 - Training: 3437batch [28:42,  1.98batch/s, Batch Loss=0.0284, Avg Loss=0.0738, Time Left=0.19 min]
Epoch 3/3 - Training: 3437batch [28:43,  1.98batch/s, Batch Loss=0.0005, Avg Loss=0.0738, Time Left=0.18 min]
Epoch 3/3 


Epoch 3/3 Results:
Train Loss: 0.0738
Validation Loss: 0.0757, Accuracy: 0.9740



('./phobert-finetuned-vietnamese\\tokenizer_config.json',
 './phobert-finetuned-vietnamese\\special_tokens_map.json',
 './phobert-finetuned-vietnamese\\vocab.txt',
 './phobert-finetuned-vietnamese\\bpe.codes',
 './phobert-finetuned-vietnamese\\added_tokens.json')

In [161]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel


# Average pooling function
def average_pool(last_hidden_states: torch.Tensor,
                 attention_mask: torch.Tensor) -> torch.Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Function to get embeddings using saved tokenizer and model
def get_embeddings2(texts, model_dir):  # `texts` is a list of sentences
    # Load the tokenizer and model from the local directory
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModel.from_pretrained(model_dir)

    # Tokenize the input texts
    batch_dict = tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**batch_dict)

    # Average pooling
    last_hidden_states = outputs.last_hidden_state
    attention_mask = batch_dict['attention_mask']
    embeddings = average_pool(last_hidden_states, attention_mask)

    # Normalize embeddings
    embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

    return embeddings.numpy()

# Function to precompute and save embeddings for a list of food names
def precompute_embeddings(food_list, model_dir, save_path):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModel.from_pretrained(model_dir)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize and compute embeddings in batches
    batch_size = 64  # Adjust batch size for memory usage
    all_embeddings = []
    for i in range(0, len(food_list), batch_size):
        batch_texts = food_list[i:i + batch_size]
        batch_dict = tokenizer(batch_texts, max_length=64, padding=True, truncation=True, return_tensors='pt')
        batch_dict = {k: v.to(device) for k, v in batch_dict.items()}
        
        with torch.no_grad():
            outputs = model(**batch_dict)
        
        # Average pooling
        last_hidden_states = outputs.last_hidden_state
        attention_mask = batch_dict['attention_mask']
        embeddings = average_pool(last_hidden_states, attention_mask)
        
        # Normalize embeddings
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        all_embeddings.append(embeddings.cpu().numpy())
    
    # Concatenate all embeddings and save to file
    all_embeddings = np.vstack(all_embeddings)
    np.save(save_path, all_embeddings)  # Save embeddings as .npy file
    return all_embeddings

df = pd.read_csv("./dataset.txt", sep='|', on_bad_lines='skip', header=None)
df = df[[0,2,4]]
df.columns = ['description','name','labels']
df['labels'] = df['labels'].astype(int)

# Example: Precompute and save embeddings
model_dir = './phobert-finetuned-vietnamese'
food_list = df['name'].unique().tolist()  # Extract unique food names
save_path = './food_embeddings.npy'
precompute_embeddings(food_list, model_dir, save_path)

array([[-0.01935733,  0.01708665,  0.03139011, ..., -0.01061608,
         0.02952815,  0.05609883],
       [-0.0293919 ,  0.01535334,  0.0241307 , ..., -0.00761916,
         0.04555051,  0.0392148 ],
       [-0.04319261,  0.01367596,  0.01487689, ..., -0.0075201 ,
         0.03285275,  0.01059048],
       ...,
       [-0.03481395,  0.0213473 ,  0.0120395 , ..., -0.01107377,
         0.05397564,  0.02819412],
       [-0.02788912,  0.01280272,  0.02456283, ..., -0.00767387,
         0.04115546,  0.04625097],
       [-0.0420851 ,  0.03776307, -0.00973398, ...,  0.02489198,
         0.0119226 ,  0.014562  ]], dtype=float32)

In [162]:
from sklearn.metrics.pairwise import cosine_similarity


# Function to find similar food names using precomputed embeddings
def get_similar_vietnamese_food_fast(food_name, food_list, embeddings_path, model_dir, threshold=0.6, limit=10):
    # Load precomputed embeddings
    food_embeddings = np.load(embeddings_path)
    
    # Compute embedding for the input food name
    input_embedding = get_embeddings2([food_name], model_dir)
    
    # Compute cosine similarity
    similarities = cosine_similarity(input_embedding, food_embeddings)[0]
    
    # Pair similarities with corresponding food names
    indexed_scores = [(i, score) for i, score in enumerate(similarities) if score > threshold]
    
    # Sort scores in descending order
    sorted_scores = sorted(indexed_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top `limit` most similar items
    top_results = sorted_scores[:limit]
    
    # Retrieve the corresponding food names
    results = [(food_list[i], score) for i, score in top_results]
    
    return results


In [169]:
# Test function
def test_local_model_fast():
    # Define the saved model directory and embeddings path
    model_dir = './phobert-finetuned-vietnamese'
    embeddings_path = './food_embeddings.npy'
    
    # Example food descriptions
    food_names = ["nước ép cam tươi nguyên chất",
                  "nuoc ep cam nguyen chat",
                  "nuoc ep",
                  "nước cam",
                  "nuoc cam",
                  "cơm đùi gà nướng",
                  "cơm dui ga",
                  "tra sữa",
                  "trà sữa phúc long"]

    # Similarity threshold
    threshold = 0.6

    for food_name in food_names:
        # Get similar food names
        similar_foods = get_similar_vietnamese_food_fast(food_name, df['name'].unique().tolist(), embeddings_path, model_dir, threshold, limit=10)

        # Print results
        print(f"Input Food Description: {food_name}")
        print("Top Similar Foods:")
        for food, score in similar_foods:
            print(f"+ {food} (Similarity: {score:.4f})")
        print()
test_local_model_fast()

Input Food Description: nước ép cam tươi nguyên chất
Top Similar Foods:
+ nước ép cam tươi nguyên chất (Similarity: 1.0000)
+ nước ép lê táo nguyên chất (Similarity: 0.9998)
+ nước ép táo dứa nguyên chất (Similarity: 0.9997)
+ nước ép cam nguyên chất (Similarity: 0.9997)
+ nước chanh tươi nguyên chất (Similarity: 0.9996)
+ nước ép lựu đỏ nguyên chất (Similarity: 0.9995)
+ nước ép thơm nguyên chất (Similarity: 0.9995)
+ nước ép cam nguyên chất chai (Similarity: 0.9993)
+ nước ép cà rốt nguyên chất (Similarity: 0.9993)
+ nước ép chai dưa lưới (Similarity: 0.9993)

Input Food Description: nuoc ep cam nguyen chat
Top Similar Foods:
+ combo sôt măm toi vưa (Similarity: 0.9951)
+ twister hoac fanta lon (Similarity: 0.9927)
+ mega combo (Similarity: 0.9927)
+ rice meal (Similarity: 0.9926)
+ tra tăc chém gió (Similarity: 0.9922)
+ sprite lon (Similarity: 0.9921)
+ almond tuile (Similarity: 0.9921)
+ chicken sandwich (Similarity: 0.9920)
+ cold whisk matcha latte (Similarity: 0.9917)
+ olong s

In [17]:
from transformers import RobertaModel
import torch
from functools import partial
import math

class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, r, alpha):
        super().__init__()
        self.r = r
        self.alpha = alpha

        # Initialize A to kaiming uniform following code: https://github.com/microsoft/LoRA/blob/main/loralib/layers.py
        self.A = torch.nn.Parameter(torch.empty(r, in_dim))
        # Initialize B to zeros.
        self.B = torch.nn.Parameter(torch.empty(out_dim, r))
        torch.nn.init.kaiming_uniform_(self.A, a=math.sqrt(5))
        torch.nn.init.zeros_(self.B)

        self.scaling = self.alpha / self.r

    def forward(self, x):
        x = self.scaling * (x @ self.A.transpose(0, 1) @ self.B.transpose(0, 1))
        return x

class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, r, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, r, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

class RobertaWithLoRA(torch.nn.Module):
    def __init__(self, base_model_name, r=16, alpha=32, num_labels=2):
        super().__init__()
        self.base_model = RobertaModel.from_pretrained(base_model_name)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(self.base_model.config.hidden_size, num_labels)

        # Apply LoRA to the attention and intermediate layers
        self.apply_lora(self.base_model, r, alpha)

    def apply_lora(self, model, r, alpha):
        assign_lora = partial(LinearWithLoRA, r=r, alpha=alpha)

        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                # Replace the linear layer with a LoRA-enhanced version
                setattr(
                    module,
                    "forward",
                    assign_lora(module).forward,
                )

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # Use [CLS] token representation
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits


In [21]:
from transformers import AutoTokenizer

# Load tokenizer and model from the local directory
model_dir = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Initialize the LoRA-enhanced model
num_labels = 2  # Change according to your task
lora_r = 8
lora_alpha = lora_r * 2
lora_model = RobertaWithLoRA(base_model_name=model_dir, r=lora_r, alpha=lora_alpha, num_labels=num_labels)


In [22]:
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Trainable parameters without LoRA:", count_trainable_params(model))
print("Trainable parameters with LoRA:", count_trainable_params(lora_model))


Trainable parameters without LoRA: 135590402
Trainable parameters with LoRA: 134999810


In [ ]:
# Train the LoRA-enhanced model
trainer = PhoBERTTrainer(
    model=lora_model, 
    tokenizer=tokenizer, 
    train_dataset=train_dataset, 
    val_dataset=val_dataset, 
    batch_size=8, 
    lr=5e-5,
    device="cuda" if torch.cuda.is_available() else "cpu"
)
trainer.fine_tune(epochs=3)

# Save the fine-tuned model
lora_model.phobert.save_pretrained("./phobert-finetuned-lora-vietnamese")
tokenizer.save_pretrained("./phobert-finetuned-lora-vietnamese")

D:\envs\tf26\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Starting Epoch 1/3



Epoch 1/3 - Training:   0%|                                                                | 0/6785 [00:00<?, ?batch/s]